In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import glob
import pickle
from moviepy.editor import VideoFileClip

%matplotlib inline

In [9]:
# Load camera calibration data
pickle_data = pickle.load(open("camera_cal/calibration_pickle.p", "rb"))
mtx = pickle_data['mtx']
dist = pickle_data['dist']

In [10]:
def abs_sobel_thresh(img, orient='x', thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    if orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def color_threshold(image, sthresh=(0, 255), vthresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    s_channel = hls[:, :, 2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0]) & (s_channel < sthresh[1])] = 1

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    v_channel = hsv[:, :, 2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= vthresh[0]) & (v_channel < vthresh[1])] = 1

    output = np.zeros_like(s_channel)
    output[(s_binary == 1) & (v_binary == 1)] = 1
    return output

def thresholdImage(image):
    gradx = abs_sobel_thresh(image, orient='x', thresh=(12,100))
    grady = abs_sobel_thresh(image, orient='y', thresh=(25,100))
    c_binary = color_threshold(image, sthresh=(100, 255), vthresh=(50, 255))
    combined = np.zeros_like(c_binary)
    combined[((gradx == 1) & (grady == 1)) | (c_binary == 1)] = 255
    return combined

# Do thresholding
#threshold_images = [thresholdImage(i) for i in undistort_images]

In [11]:
def plotTransformImages(imgs, titles, dots):
    fig = plt.figure(figsize=(14, 10))
    for i, img in enumerate(imgs):
        plt.subplot(1, 2, i + 1)
        plt.title(titles[i])
        plt.imshow(img, cmap='gray')
        plt.plot(dots[i][0][0], dots[i][0][1], 'r.', ms=12) # top right
        plt.plot(dots[i][1][0], dots[i][1][1], 'r.', ms=12) # bottom right
        plt.plot(dots[i][2][0], dots[i][2][1], 'r.', ms=12) # bottom left
        plt.plot(dots[i][3][0], dots[i][3][1], 'r.', ms=12) # top left
    plt.show()
    
def transformImage(image, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    return warped

# Transformation coordinates
src = np.float32([[693, 455],[1075, 700],[230, 700],[588, 455]])
dst = np.float32([[950, 0],[950, 720],[250, 720],[250, 0]])

# Do transformation
#transformed_images = [transformImage(i,src,dst) for i in threshold_images]

In [15]:
def findWindowCentroids(img, window_width, rows, margin=40, last_centroids=[0, 0], DEBUG=False):
    window_centroids = []
    window_height = int(img.shape[0] / rows)
    half_image_width = int(img.shape[1]/2)
    left_lane_x = last_centroids[0]
    right_lane_x = last_centroids[1]
    left_start = 0 if left_lane_x == 0 else left_lane_x - margin
    left_end = half_image_width - 1 if left_lane_x == 0 else left_lane_x + margin
    right_start = half_image_width if right_lane_x == 0 else right_lane_x - margin
    right_end = img.shape[1] - 1 if right_lane_x == 0 else right_lane_x + margin
    
    for i, y_window_pos in enumerate(range(img.shape[0], 0, -window_height)):
        row_image = img[y_window_pos - window_height: y_window_pos]
        conv_signal = np.convolve(np.ones(window_width), np.sum(row_image, axis=0))
            
        if window_centroids:
            left_lane_x, right_lane_x = window_centroids[-1]
            left_start = left_lane_x - margin
            left_end = left_lane_x + margin
            right_start = right_lane_x - margin
            right_end = right_lane_x + margin

        if np.max(conv_signal[left_start:left_end]) > 0:
            left_lane_x = np.argmax(conv_signal[left_start:left_end]) + left_start
        
        if np.max(conv_signal[right_start:right_end]) > 0:
            right_lane_x = np.argmax(conv_signal[right_start:right_end]) + right_start
        
        window_centroids.append((left_lane_x,right_lane_x))

    return np.array(window_centroids) - window_width / 2

def updateMask(mask, window_width, rows, image_shape, center, level):
    window_height = image_shape[0] / rows
    height_start = int(image_shape[0] - (level + 1) * window_height)
    height_end = int(image_shape[0] - level * window_height)
    width_start = max(0, int(center - window_width / 2))
    width_end = min(int(center + window_width / 2), image_shape[1])
    mask[height_start:height_end, width_start:width_end] = 255

def getWindowOverlayImage(image, window_centroids, window_width, rows):
    zeros = np.zeros_like(image)
    mask = np.zeros_like(image)
    
    # Go through each level and update window mask
    for i, centroid in enumerate(window_centroids):
        updateMask(mask, window_width, rows, image.shape, centroid[0], i)
        updateMask(mask, window_width, rows, image.shape, centroid[1], i)

    # Make window pixels green
    windows = np.array(cv2.merge((zeros, np.array(mask), zeros)), np.uint8)
    # Overlay the orignal road image with window results
    return cv2.addWeighted(cv2.cvtColor(image, cv2.COLOR_GRAY2RGB), 0.5, windows, 0.5, 0.0)

def getLaneImages(images, window_width, window_rows):
    output_images = []
    output_centroids = []
    for image in images:
        centroids = findWindowCentroids(image, window_width, window_rows)
        output_images.append(getWindowOverlayImage(image, centroids, window_width, window_rows))
        output_centroids.append(centroids)
    return output_images, output_centroids

# Detect lanes on all birds eye test images
#detected_lane_images, detected_centroids = getLaneImages(transformed_images, 50, 8)

In [16]:
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
def textOverlay(img, res_yvals, yvals, leftx, left_fitx, right_fitx):
    curve_fit_cr = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix, np.array(leftx, np.float32) * xm_per_pix, 2)
    curverad = ((1 + (2 * curve_fit_cr[0] * yvals[-1] * ym_per_pix + curve_fit_cr[1])**2)**1.5) / np.absolute(2*curve_fit_cr[0])
    camera_center = (left_fitx[-1] + right_fitx[-1]) / 2
    center_diff = (camera_center-img.shape[1]/2) * xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pos = 'right'
    cv2.putText(img, 'Radius of Curvature = ' + str(round(curverad, 3)) + 'm', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 
        (255,255,255), 2)
    cv2.putText(img, 'Vehicle is ' + str(abs(round(center_diff,3))) + 'm ' + side_pos + ' of center', (50,100), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    

def drawLane(img, centroids, transformed):
    img = img.copy()
    leftx = [i[0] for i in centroids]
    rightx = [i[1] for i in centroids]
    
    yvals = range(img.shape[0])
    res_yvals = np.arange(img.shape[0]-(img.shape[0]/16), 0, -img.shape[0] / len(centroids) )
    
    # f(x) = Ax**2 + Bx + C
    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = np.array(left_fit[0] * yvals * yvals +  left_fit[1] * yvals + left_fit[2], np.int32)

    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = np.array(right_fit[0] * yvals * yvals +  right_fit[1] * yvals + right_fit[2], np.int32)
    
    textOverlay(img, res_yvals, yvals, leftx, left_fitx, right_fitx)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warp_zero = np.zeros_like(transformed).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx, yvals]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, yvals])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    # Combine the result with the original image

    return cv2.addWeighted(img, 1, newwarp, 0.3, 0)

#lane_images = [drawLane(i, c, t) for i, c, t in zip(undistort_images, detected_centroids, transformed_images)]

In [14]:
min_left_lane = []
max_left_lane = []
min_right_lane = []
max_right_lane = []

def processImage(image):
    img_undistorted = cv2.undistort(image, mtx, dist, None, mtx)
    img_thresholded = thresholdImage(img_undistorted)
    img_transformed = transformImage(img_thresholded, src, dst)
    centroids = findWindowCentroids(img_transformed, 50, 8)
    
    min_left_lane.append(centroids[:,0].min())
    max_left_lane.append(centroids[:,0].max())
    min_right_lane.append(centroids[:,1].min())
    max_right_lane.append(centroids[:,1].max())
    
    return drawLane(img_undistorted, centroids, img_transformed)
    
input_video = './project_video.mp4'
output_video = './videos/out_project_video.mp4'

clip = VideoFileClip(input_video)
video_clip = clip.fl_image(processImage)
video_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video ./videos/out_project_video.mp4
[MoviePy] Writing video ./videos/out_project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [02:29<00:00,  8.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/out_project_video.mp4 



In [ ]:

template = np.array(cv2.merge((zeros_channel, template, zero_channel)), np.uint8)
warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8)
resutls = cv2.addWeighted(warpage, 1, template, 0.5, 0.0)

# Fit lane boundaries to the left, right, and center positions

yvals = range(0, warped.shape[0])

res_yvals = np.arrange(warped.shape[0] - (window_height / 2), 0, -window_height)

left_fit = np.polyfit(res_yvals, leftx, 2)
left_fitx = left_fit[0] * yvals**2 + left_fit[1] * yvals + left_fit[2]
left_fitx = np.array(left_fitx, np.int32)

right_fit = np.polyfit(res_yvals, rightx, 2)
right_fitx = right_fit[0] * yvals**2 + right_fit[1] * yvals + right_fit[2]
right_fitx = np.array(right_fitx, np.int32)

left_lane = np.array(list(zip(np.concatenate((left_fitx - window_width / 2, left_fitx[::-1]) + window_width / 2), axis = 0), np.concatenate((yvals))))
right_lane = np.array(list(zip(np.concatenate((right_fitx - window_width / 2, right_fitx[::-1]) + window_width / 2), axis = 0), np.concatenate((yvals))))
middle_marker = np.array(list(zip(np.concatenate((right_fitx - window_width / 2, left_fitx[::-1]) + window_width / 2), axis = 0), np.concatenate((yvals))))

road = np.zeros_like(img)
road_bkg = np.zeros_like(img)

cv2.fillPoly(road, [left_lane], color = [255, 0, 0])
cv2.fillPoly(road, [right_lane], color = [0, 0, 255])
cv2.fillPoly(road_bkg, [left_lane], color = [255, 255, 255])
cv2.fillPoly(road_bkg, [right_lane], color = [255, 255, 255])

road_warped = cv2.warpPerspective(road, Minv, img_size, flags = cv2.INTER_LINEAR)
road_warped_bkg = cv2.warpPerspective(road, Minv, img_size, flags = cv2.INTER_LINEAR)

base = cv2.addWeighted(img, 1.0, road_warped_bkg, 1.0, 0.0)
result = cv2.addWeighted(base, 1.0, road_warped, 1.0, 0.0)

ym_per_pix = 30 / 720 # meters per pixel in y dimension
xm_per_pix = 3.7 / 700 # meters per pixel in x dimension

curve_fit_cr = np.polyfit(np.arry(res_yvals, np.float32) * (ym_per_pix), np.array(leftx, np.float32) * xm_per_pix, 2)
curverad = ((1 + (2 * curve_fit_cr[0] * y_vals[-1] * ym_per_pix + curve_fit_cr[1])**2)**1.5) / np.absolute(2 * curve_fit_cr[0])


# Calculate the offset of the car on the road
camera_center = (left_fitx[-1] + right_fitx[-1]) / 2
center_diff = (camera_center-warped.shape[1] / 2) * xm_per_pix

side_pos = 'left'

if center_diff <= 0:
    side_pos = 'right'
    
# draw the text showing curvatue, offset, and speed
# Annotate image with lane curvature estimates
cv2.putText(result, 'Left Lane Radius: ' + str(round(curverad, 3)) + '(m)', (50,50), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)
cv2.putText(result, 'Vehicle Position: ' + str(abs(round(center_diff, 3))) + '(m)' + sid_pos + 'of center', (50,100), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)


write_name = './output_images/tracked' + str(idx) + '.jpg'
cv2.imwrite(write_name, result)

In [ ]:
# Read in images from a folder and perform an operation on them

images = glob.glob('./test_images/test*.jpg')

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    result = img
    
    write_name = './test_images/tracked' + str(idx) + '.jpg'
    cv2.imwrite(write_name,result)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2

# Read in a thresholded image
warped = mpimg.imread('warped_example.jpg')
# window settings
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
	    # convolve the window into the vertical slice of the image
	    image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
	    conv_signal = np.convolve(window, image_layer)
	    # Find the best left centroid by using past left center as a reference
	    # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
	    offset = window_width/2
	    l_min_index = int(max(l_center+offset-margin,0))
	    l_max_index = int(min(l_center+offset+margin,image.shape[1]))
	    l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
	    # Find the best right centroid by using past right center as a reference
	    r_min_index = int(max(r_center+offset-margin,0))
	    r_max_index = int(min(r_center+offset+margin,image.shape[1]))
	    r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
	    # Add what we found for that layer
	    window_centroids.append((l_center,r_center))

    return window_centroids

window_centroids = find_window_centroids(warped, window_width, window_height, margin)

# If we found any window centers
if len(window_centroids) > 0:

    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    # Go through each level and draw the windows 	
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
	    l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
	    r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
	    # Add graphic points from window mask here to total pixels found 
	    l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
	    r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
# If no window centers found, just display orginal road image
else:
    output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

# Display the final results
plt.imshow(output)
plt.title('window fitting results')
plt.show()


In [ ]:
def detect_lines(binary_warped, left_fit, right_fit):
    
    # Define positions for all non zero pixels
    nonzero = binary_wapred.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])    
    
    margin = 100
    
    # left_curverad, right_curverad, center_dist = (0, 0, 0)
    
    # Choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    left_lane_inds = ((nonzerox > (left_fit[0] * (nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0] * (nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin))) 
    
    # Extract left and right line pixel positions
    leftx = nonzerox[l_lane_inds]
    lefty = nonzeroy[l_lane_inds] 
    rightx = nonzerox[r_lane_inds]
    righty = nonzeroy[r_lane_inds]
    
    left_fit,left_res,_,_,_ = np.polyfit(lefty, leftx, 2, full=True)
    right_fit,right_res,_,_,_ = np.polyfit(righty, rightx, 2, full=True)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    
    left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]
    
    y_eval = warped_binary.shape[0]    
    
    # Compute convsions between pixels and real-world dimensions
    ym_per_pix = 30 / 720 # meters per pixel in y dimension
    xm_per_pix = 3.7 / 700 # meters per pixel in x dimension
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty * ym_per_pix, left_fitx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, right_fitx * xm_per_pix, 2)
    
    #left_fit_cr = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2)
    #right_fit_cr = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2)    
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2 * right_fit_cr[0])
    
    # Compute car position
    m_car = binary_warped.shape[1] / 2
    m_lane = (left_fitx[0] + right_fitx[0]) / 2
    offset_right_from_center_m = (m_lane - m_car) * xm_per_pix
    
    # Compute radius of curvature in meters
    avg_radius_meters = np.mean([left_curverad, right_curverad])
    
    #return left_curverad, right_curverad, left_fitx, right_fitx, offset_right_from_center_m
    return left_fit, right_fit, np.sqrt(left_fit[1]/len(leftx)), np.sqrt(right_fit[1]/len(rightx)), left_curverad, right_curverad, None

In [ ]:
img_test1 = mpimg.imread('test_images/test1.jpg')

dist_pickle = pickle.load(open("camera_cal/calibration_pickle.p", "rb"))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

#undistorted1 = undistort(img_test1)
undistorted1 = cv2.undistort(img_test1, mtx, dist, None, mtx)
binary_warped1 = topdown(pipeline_color(undistorted1, s_thresh=s_thresh, sx_thresh=sx_thresh, sobel_kernel=9))

l_fit, r_fit, l_res, r_res, l_curverad, r_curverad, _ = sliding_windows(binary_warped1)

annotated_test = draw_lane(undistorted1, binary_warped1, l_fit, r_fit, _, _)

In [ ]:
# Define a color threshold function with HLS and HSV and test it.

def color_lab(img):
    
    LAB = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    LAB_L = LAB[:,:,0]
    LAB_A = LAB[:,:,1]
    LAB_B = LAB[:,:,2]
    
    return LAB, LAB_L, LAB_A, LAB_B

In [ ]:
def color_lab(img, thresh=(190,255)):
    
    # Convert to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    
    lab_b = lab[:,:,2]
    
    # don't normalize if there are no yellows in the image
    if np.max(lab_b) > 150:
        lab_b = lab_b * (255 / np.max(lab_b))
    
    # 2) Apply a threshold to the L channel
    binary_output = np.zeros_like(lab_b)
    binary_output[((lab_b > thresh[0]) & (lab_b <= thresh[1]))] = 1
    
    # 3) Return a binary image of threshold result
    return binary_output